In [3]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from src.db import DataBase
from src.utils_general import beeps
from src.utils_date import add_days
from src.utils_stocks import get_ls_sym
from src.utils_stocks import get_df_prices
from src.utils_stocks import suppress_stdout
try:
    with open('dir.txt') as f: dir_db = json.load(f)['dir_db']
except:
    dir_db = ''
    assert dir_db, 'Please provide a directory to save the sqlite database, in the "dir_db" variable!'

def db_remove_dups_prices_m(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_m
         WHERE DATE(datetime)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_m
                WHERE DATE(datetime)>='{}'
                GROUP BY sym, datetime)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_prices_d(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_d
         WHERE DATE(date)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_d
                WHERE DATE(date)>='{}'
                GROUP BY sym, date)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_stocks(db):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM stocks
         WHERE ROWID NOT IN
               (SELECT MAX(ROWID)
                  FROM stocks
                 GROUP BY sym)
    '''
    db.execute(q)

ls_init_str = [
    #prices_m
    '''CREATE TABLE IF NOT EXISTS prices_m(
        sym TEXT
        ,datetime TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER
        ,is_reg_hours INTEGER)''',
    
    '''CREATE INDEX IF NOT EXISTS index_prices_m_all
        ON prices_m(sym, date(datetime), is_reg_hours)''',

    '''CREATE INDEX IF NOT EXISTS index_prices_m_date
        ON prices_m(date(datetime))''',

    #prices_d
    '''CREATE TABLE IF NOT EXISTS prices_d(
        sym TEXT
        ,date TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER)''',
   
    '''CREATE INDEX IF NOT EXISTS index_prices_d_date
        ON prices_d(sym, date(date))''',
    
    #stocks
    '''CREATE TABLE IF NOT EXISTS stocks(
        sym TEXT
        ,long_name TEXT
        ,sec TEXT
        ,ind TEXT
        ,quote_type TEXT
        ,fund_family TEXT
        ,summary TEXT
        ,timestamp TEXT)''',

    '''CREATE INDEX IF NOT EXISTS index_stocks
        ON stocks(sym, quote_type)''',
    
    #stocks_error
    '''CREATE TABLE IF NOT EXISTS stocks_error(
        sym TEXT)''',
    
    #proba
    '''CREATE TABLE IF NOT EXISTS proba(
        sym TEXT
        ,datetime TEXT
        ,my_index INTEGER
        ,proba REAL
        ,datetime_update TEXT)''',
]

db = DataBase(ls_init_str, dir_db)

# Update table: stocks

In [4]:
def get_df_info(sym):
    dt_info = yf.Ticker(sym).info
    dt_info['timestamp'] = datetime.datetime.now()
    dt_info['sector'] = dt_info.get('sector')
    dt_col = {
        'symbol':'sym',
        'longName':'long_name',
        'sector':'sec',
        'industry':'ind',
        'quoteType':'quote_type',
        'fundFamily':'fund_family',
        'longBusinessSummary':'summary',
        'timestamp':'timestamp',
    }
    dt_info = {key:dt_info.get(key) for key in dt_col}
    df = pd.DataFrame([dt_info])
    df = df.rename(columns=dt_col)
    return df

#get ls_sym
ls_sym = get_ls_sym()
q = '''
    SELECT sym FROM STOCKS UNION ALL
    SELECT sym FROM stocks_error
    --SELECT sym FROM stocks WHERE summary IS NOT NULL
'''
ls_sym_exclude = pd.read_sql(q, db.conn)['sym'].to_list()
ls_sym = [x for x in ls_sym if x not in ls_sym_exclude]
#extract and load
ls_df = []
for i, sym in enumerate(ls_sym):
    try:
        if (i+1)%50==0 and ls_df:
            df = pd.concat(ls_df)
            print('Saving...')
            df.to_sql('stocks', db.conn, if_exists='append', index=0)
            ls_df = []
        print(sym, i+1, len(ls_sym), end=' ')
        df_info = get_df_info(sym)
        ls_df.append(df_info)
        print('Done.')
    except Exception as e:
        print(f'Skipped... ({type(e).__name__}: {str(e)})')
        df = pd.DataFrame([{'sym':sym}])
        df.to_sql('stocks_error', db.conn, if_exists='append', index=0)
#load remaining
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('stocks', db.conn, if_exists='append', index=0)
#remove duplicates
db_remove_dups_stocks(db)
beeps()

Extracting symbols from URL 1/2...
Extracting symbols from URL 2/2...
SNRHU 1 69 Done.
KINZU 2 69 Done.
ATA.U 3 69 Skipped... (KeyError: 'regularMarketOpen')
QQC 4 69 Done.
IIIIU 5 69 Skipped... (ImportError: BeautifulSoup4 (bs4) not found, please install it)
GFX.U 6 69 Skipped... (KeyError: 'regularMarketOpen')
TMPM 7 69 Skipped... (KeyError: 'regularMarketOpen')
AIRC 8 69 Done.
FDMT 9 69 Done.
RICE 10 69 Done.
RVPH 11 69 Done.
BFIIW 12 69 Done.
XOMAP 13 69 Skipped... (ImportError: BeautifulSoup4 (bs4) not found, please install it)
KBUY 14 69 Done.
WNW 15 69 Done.
CLVR 16 69 Skipped... (ImportError: BeautifulSoup4 (bs4) not found, please install it)
AJAX 17 69 Skipped... (ImportError: BeautifulSoup4 (bs4) not found, please install it)
WISH 18 69 Done.
MTACU 19 69 Done.
SCPS 20 69 Done.
TEKKW 21 69 Skipped... (KeyError: 'regularMarketOpen')
TVACU 22 69 Done.
BTWNW 23 69 Done.
MUDSU 24 69 Done.
DCRBW 25 69 Done.
WWOW 26 69 Done.
BHSEW 27 69 Done.
FICS 28 69 Done.
SVOKU 29 69 Skipped... 

# Update table: prices_m

In [5]:
def get_ls_df_prices_m(ls_sym, ls_date_str, ls_sym_exclude=[]):
    count = 0
    count_e = 0
    ls_df = []
    for i, sym in enumerate(ls_sym):
        count+=1
        try:
            if i%100==0:
                print(i+1, len(ls_sym))
            df = get_df_prices(sym, ls_date_str[0], ls_date_str[-1])
            len_unique_dates = len(df['datetime'].dt.date.unique())
            len_candles_per_day = df.shape[0]/len_unique_dates
            if len_unique_dates<len(ls_date_str):
                print('Skip: Incomplete data', sym, len_unique_dates)
            elif len_candles_per_day<200:
                print('Skip: Not enough candles per day', sym, len_candles_per_day)
            else:
                ls_df.append(df)
        except Exception as e:
            print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
            count_e+=1
    print(count, count_e, round(count_e/count, 3))
    return ls_df

# get max date present
q = '''
    SELECT DATE(MAX(datetime))
      FROM prices_m
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# get missing dates
df = yf.download('IBM', period='1y', interval='1d', progress=0).reset_index()
df['Date']= df['Date'].astype('str')
ls_date_str = df[df['Date']>max_date_str]['Date'].to_list()
assert ls_date_str, 'No dates to extract!'
print('Extracing prices in ', ls_date_str)
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract and load
ls_df = get_ls_df_prices_m(ls_sym, ls_date_str)
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('prices_m', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_m(db, ls_date_str[0])
    beeps()

Extracing prices in  ['2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18']
1 4344
Error: GPAQ, KeyError: 'Datetime'
Error: FMCI, KeyError: 'Datetime'
Error: MJCO, KeyError: 'Datetime'
Skip: Not enough candles per day PBCTP 41.2
Error: WINS, KeyError: 'Datetime'
Error: MYL, KeyError: 'Datetime'
Error: ZN, KeyError: 'Datetime'
Error: SPAQ, KeyError: 'Datetime'
Error: LVGO, KeyError: 'Datetime'
Error: CNXM, KeyError: 'Datetime'
Error: TRWH, KeyError: 'Datetime'
Skip: Not enough candles per day BBDO 21.0
Skip: Not enough candles per day VLYPO 7.8
Skip: Not enough candles per day VBLT 193.8
Skip: Not enough candles per day NSYS 14.2
Error: RUBI, KeyError: 'Datetime'
Skip: Not enough candles per day UBCP 9.6
Error: CVTI, KeyError: 'Datetime'
Error: GARS, KeyError: 'Datetime'
Skip: Not enough candles per day DRQ 194.4
Error: MINI, KeyError: 'Datetime'
Error: PPHI, KeyError: 'Datetime'
Error: ORSNU, KeyError: 'Datetime'
Error: WMGI, KeyError: 'Datetime'
Error: BITA, KeyError: 

Skip: Incomplete data ACEVU 4
Skip: Not enough candles per day RMRM 59.2
Skip: Not enough candles per day WRN 128.6
Skip: Not enough candles per day LGHL 127.0
Skip: Not enough candles per day GFNCP 8.8
Skip: Not enough candles per day DSM 136.0
Skip: Not enough candles per day TCBIP 15.0
Skip: Not enough candles per day CEN 44.8
Skip: Not enough candles per day ANGO 174.6
Skip: Not enough candles per day KFRC 113.4
Skip: Not enough candles per day FCCO 24.4
Skip: Not enough candles per day ABST 74.6
Skip: Not enough candles per day GIFI 121.6
Skip: Not enough candles per day ATAX 157.8
Skip: Not enough candles per day TOWN 137.2
Skip: Not enough candles per day SCVX 57.4
Skip: Not enough candles per day LATN 7.8
Skip: Not enough candles per day IMXI 174.6
Skip: Not enough candles per day NOVSU 5.8
Skip: Not enough candles per day PBLA 101.2
Skip: Not enough candles per day KRKR 29.4
Skip: Not enough candles per day OFS 48.2
Skip: Not enough candles per day RYI 165.6
Skip: Not enough c

Skip: Not enough candles per day JG 185.0
Skip: Not enough candles per day EXD 32.4
Skip: Not enough candles per day DLNG 137.8
Skip: Not enough candles per day MBNKP 10.6
Skip: Not enough candles per day CCAC 168.0
Skip: Not enough candles per day FFG 68.8
Skip: Not enough candles per day FBRX 95.8
Skip: Not enough candles per day SHBI 46.4
Skip: Not enough candles per day CHAQ 8.2
Skip: Not enough candles per day RPTX 168.4
Skip: Not enough candles per day BWB 78.0
Skip: Not enough candles per day DYAI 146.0
Skip: Not enough candles per day ACET 112.2
Skip: Not enough candles per day SANW 81.8
Skip: Not enough candles per day ANAT 55.4
Skip: Not enough candles per day CPS 178.6
Skip: Not enough candles per day FWONA 166.8
Skip: Not enough candles per day PFN 142.6
Skip: Not enough candles per day SJW 92.6
Skip: Not enough candles per day SMHI 172.4
Skip: Not enough candles per day GLP 113.8
Skip: Not enough candles per day NGVT 194.4
Skip: Not enough candles per day AEF 41.8
Skip: No

Skip: Not enough candles per day PFC 124.8
Skip: Not enough candles per day CBPO 186.4
Skip: Not enough candles per day GUT 78.0
Skip: Not enough candles per day INTT 61.0
Skip: Not enough candles per day CCX 172.8
Skip: Not enough candles per day DAIO 81.4
Skip: Not enough candles per day MSGS 190.6
Skip: Not enough candles per day ALGT 158.4
Skip: Not enough candles per day MUJ 51.6
Skip: Not enough candles per day EDF 163.6
Skip: Not enough candles per day HURC 22.2
Skip: Not enough candles per day NEXT 154.4
Skip: Not enough candles per day AMPH 161.4
Skip: Not enough candles per day AZZ 178.4
Skip: Not enough candles per day ARKR 69.0
Skip: Not enough candles per day ALPN 167.8
Skip: Not enough candles per day PFIE 38.4
Skip: Not enough candles per day TUSK 182.8
Skip: Not enough candles per day Y 92.6
Skip: Not enough candles per day PEO 133.2
Skip: Not enough candles per day SSBI 14.2
Skip: Not enough candles per day BROG 15.6
Skip: Not enough candles per day MFNC 25.2
Skip: Inc

Skip: Not enough candles per day GLO 90.6
Skip: Not enough candles per day KEQU 10.0
Skip: Incomplete data PIH 3
Skip: Not enough candles per day BASI 104.0
Skip: Incomplete data SLMBP 4
Skip: Not enough candles per day NUM 23.0
Skip: Not enough candles per day NYMTN 22.2
Skip: Not enough candles per day PT 79.8
Skip: Not enough candles per day CLW 167.4
Skip: Not enough candles per day HEQ 105.8
Skip: Not enough candles per day CHPM 6.4
Skip: Not enough candles per day STIM 184.0
Skip: Not enough candles per day JRS 111.4
1701 4344
Skip: Not enough candles per day OFG 154.2
Skip: Not enough candles per day ONVO 183.6
Skip: Not enough candles per day LMPX 175.6
Skip: Not enough candles per day PCM 27.4
Skip: Not enough candles per day MSBI 73.2
Skip: Not enough candles per day LTRN 54.0
Skip: Not enough candles per day TYHT 39.4
Skip: Not enough candles per day CHA 185.0
Skip: Not enough candles per day WRLD 94.8
Skip: Not enough candles per day FCO 32.0
Skip: Not enough candles per da

...

Skip: Not enough candles per day CBU 174.4
Skip: Not enough candles per day ASR 118.2
Skip: Not enough candles per day PTR 197.6
Skip: Not enough candles per day GNFT 32.2
Skip: Not enough candles per day TITN 172.4
Skip: Not enough candles per day MARPS 67.4
Skip: Not enough candles per day GPRK 132.8
2501 4344
Skip: Not enough candles per day EARN 68.2
Skip: Not enough candles per day ACNB 15.0
Skip: Incomplete data LBTYB 4
Skip: Not enough candles per day BDL 3.0
Skip: Not enough candles per day DDF 28.6
Skip: Incomplete data NEN 3
Skip: Not enough candles per day CELC 17.4
Skip: Not enough candles per day GDO 68.4
Skip: Not enough candles per day GV 164.6
Skip: Not enough candles per day KTCC 34.2
Skip: Not enough candles per day CCAP 34.0
Skip: Not enough candles per day STND 22.0
Skip: Incomplete data LIVK 4
Skip: Not enough candles per day TPB 161.4
Skip: Not enough candles per day LANDP 11.6
Skip: Not enough candles per day BOMN 99.4
Skip: Not enough candles per day FIF 108.2
S

Skip: Not enough candles per day RLI 158.2
Skip: Not enough candles per day APEI 118.6
Skip: Not enough candles per day BMA 180.8
Skip: Not enough candles per day MCHX 47.2
Skip: Not enough candles per day CVCY 19.0
Skip: Not enough candles per day JHS 21.8
Skip: Not enough candles per day CABO 37.2
Skip: Not enough candles per day CNNB 5.0
Skip: Not enough candles per day FBMS 63.4
Skip: Not enough candles per day BME 72.6
Skip: Not enough candles per day CVLG 113.6
Skip: Not enough candles per day CFBI 9.0
Skip: Not enough candles per day TPHS 42.0
Skip: Not enough candles per day GSBC 63.2
Skip: Not enough candles per day PME 56.6
Skip: Not enough candles per day MBII 181.8
Skip: Not enough candles per day AIT 166.4
Skip: Not enough candles per day PDLB 14.0
Skip: Not enough candles per day CRHM 48.8
Skip: Not enough candles per day JPI 102.4
Skip: Not enough candles per day ERYP 18.2
Skip: Not enough candles per day DWSN 52.0
Skip: Not enough candles per day AXGN 153.8
Skip: Not en

Skip: Not enough candles per day BATRK 185.6
Skip: Not enough candles per day CTHR 68.8
Skip: Not enough candles per day PSF 39.2
Skip: Not enough candles per day SMBC 23.0
Skip: Not enough candles per day PLSE 133.6
Skip: Not enough candles per day MGF 68.2
Skip: Not enough candles per day BNFT 160.8
Skip: Not enough candles per day RRBI 18.0
Skip: Not enough candles per day DXR 7.4
Skip: Not enough candles per day NXN 12.0
Skip: Not enough candles per day FSRV 139.6
Skip: Incomplete data BHV 4
Skip: Not enough candles per day ESGRO 12.0
Skip: Not enough candles per day SNDE 131.8
Skip: Not enough candles per day CHMG 11.0
Skip: Not enough candles per day GBLI 16.2
Skip: Not enough candles per day SPSC 170.2
Skip: Not enough candles per day MVT 69.2
Skip: Not enough candles per day IGIC 12.0
Skip: Not enough candles per day NSIT 198.6
Skip: Not enough candles per day CALB 19.2
Skip: Not enough candles per day THM 106.0
Skip: Not enough candles per day UNB 14.2
Skip: Not enough candles

Skip: Not enough candles per day CTG 65.4
Skip: Not enough candles per day FISK 30.2
Skip: Not enough candles per day HPS 76.8
Skip: Not enough candles per day KOF 149.4
Skip: Not enough candles per day JPT 18.4
Skip: Not enough candles per day CHW 115.8
Skip: Incomplete data HMNF 4
Skip: Not enough candles per day ABG 156.4
3801 4344
Skip: Not enough candles per day NPN 3.2
Skip: Not enough candles per day AGYS 141.0
Skip: Not enough candles per day MSN 105.0
Skip: Not enough candles per day ESBA 12.4
Skip: Not enough candles per day SCL 117.6
Skip: Not enough candles per day IFN 126.2
Skip: Not enough candles per day PROV 27.4
Skip: Not enough candles per day IFMK 146.8
Skip: Not enough candles per day MN 58.2
Skip: Not enough candles per day WHLRD 3.8
Skip: Not enough candles per day OSS 122.6
Skip: Not enough candles per day KZR 186.8
Skip: Not enough candles per day FLMN 167.4
Skip: Not enough candles per day NREF 68.2
Skip: Not enough candles per day AWRE 30.0
Skip: Not enough ca

Skip: Not enough candles per day RMGBU 107.0
Skip: Not enough candles per day JTA 56.2
4201 4344
Skip: Not enough candles per day CXDO 68.0
Skip: Not enough candles per day BBN 158.6
Skip: Not enough candles per day GHVIU 69.8
Skip: Not enough candles per day HRI 157.8
Skip: Not enough candles per day PCQ 19.2
Skip: Not enough candles per day PI 197.4
Skip: Not enough candles per day FFA 51.8
Skip: Not enough candles per day WEYS 32.0
Skip: Not enough candles per day BY 66.6
Skip: Not enough candles per day VALU 6.0
Skip: Not enough candles per day UIHC 159.6
Skip: Not enough candles per day PDT 109.2
Skip: Not enough candles per day MLVF 11.4
Skip: Not enough candles per day FCBC 53.8
Skip: Not enough candles per day CIX 15.8
Skip: Not enough candles per day RBB 59.6
Skip: Not enough candles per day CLPS 151.8
Skip: Not enough candles per day GWACW 31.8
Skip: Not enough candles per day CAMT 134.4
Skip: Not enough candles per day WLDN 98.4
Skip: Not enough candles per day BCH 98.0
Skip

# Update table: prices_d

In [6]:
# get max date present
q = '''
    SELECT DATE(MAX(date))
      FROM prices_d
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# check dates
end = add_days(datetime.datetime.today().strftime('%Y-%m-%d'), 3) #today's date plus 3 days
df = yf.download('IBM', start=max_date_str, end=end, interval='1d', progress=0).reset_index()
df = df[df['Date'].astype('str')>max_date_str]
assert not df.empty, 'No dates to extract...'
print(f'Extracting from {max_date_str} plus 1, to {end}')

Extracting from 2020-12-11 plus 1, to 2020-12-22


In [7]:
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract
dt_cols = {
    'sym':'sym',
    'Date':'date',
    'Open':'open',
    'High':'high',
    'Low':'low',
    'Adj Close':'adj_close',
    'Volume':'volume',
}
count = 0
count_e = 0
ls_df = []
for i, sym in enumerate(ls_sym):
    count+=1
    try:
        if i%300==0:
            print(i+1, len(ls_sym))
        #df = yf.download(sym, start=max_date_str, end=end, interval='1d', progress=0).reset_index()
        #df = df[df['Date'].astype('str')>max_date_str]
        with suppress_stdout():
            df = yf.download(sym, period = '1mo', interval='1d', progress=0).reset_index()
        df['sym'] = sym
        df = df.rename(columns=dt_cols)
        df = df[list(dt_cols.values())]
        ls_df.append(df)
    except Exception as e:
        print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
        count_e+=1
if ls_df:
    # load
    print(count, count_e, round(count_e/count, 3))
    df = pd.concat(ls_df)
    df.to_sql('prices_d', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_d(db, max_date_str)
    beeps()

1 4344
301 4344
601 4344
901 4344
1201 4344
1501 4344
1801 4344
2101 4344
2401 4344
2701 4344
3001 4344
3301 4344
3601 4344
3901 4344
4201 4344
4344 0 0.0
Removing duplicates...


# Manual Editting

In [70]:
#execute
assert 0
q = '''
ALTER TABLE trading_days
RENAME COLUMN Date TO date
'''
q = '''
drop table trading_days
'''
q='''
ALTER TABLE prices_interday
  RENAME TO prices_d
'''
q='''
drop index index_prices_interday
'''

q='''
    UPDATE prices_m
       SET is_reg_hours = CASE
                           WHEN time(datetime) < time('09:30:00') THEN 0
                           WHEN time(datetime) > time('15:59:00') THEN 0
                           ELSE 1
                           END
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
q='''
    --UPDATE prices_m
       SET datetime = DATETIME(datetime, '-1 hours')
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
db.execute(q)
beeps()

In [8]:
#read
q = '''
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

'''
q='''
    select type, name, tbl_name, sql
    FROM sqlite_master
    WHERE type='index'
'''
q='''
    SELECT 
    --stocks.sym
           stocks.long_name
           ,stocks.sec
           ,stocks.ind
           ,stocks.quote_type
           ,stocks.fund_family
           ,stocks.summary
           ,prices_d.*
      FROM prices_d
      LEFT JOIN stocks
        ON stocks.sym = prices_d.sym
     WHERE DATE(date) = (SELECT MAX(DATE(date)) FROM prices_d WHERE sym='IBM')
       AND adj_close > 5
       AND volume > 100000
'''
df = pd.read_sql(q, db.conn)
#df.to_csv('temp.csv', index=0)
df.iloc[:]

,long_name,sec,ind,quote_type,fund_family,summary,sym,date,open,high,low,adj_close,volume
0,"CareDx, Inc",Healthcare,Diagnostics & Research,EQUITY,None,"CareDx, Inc., a precision medicine company, di...",CDNA,2020-12-04 00:00:00,61.939999,65.285004,60.020000,64.910004,467900
1,"MacroGenics, Inc.",Healthcare,Biotechnology,EQUITY,None,"MacroGenics, Inc., a biopharmaceutical company...",MGNX,2020-12-04 00:00:00,22.840000,24.190001,22.709999,24.000000,563700
2,"Advanced Energy Industries, Inc.",Industrials,Electrical Equipment & Parts,EQUITY,None,"Advanced Energy Industries, Inc. designs, manu...",AEIS,2020-12-04 00:00:00,98.080002,101.870003,98.080002,100.919998,444500
3,Calamos Convertible Opportunities and Income Fund,Financial Services,Asset Management,EQUITY,None,Calamos Convertible Opportunities and Income F...,CHI,2020-12-04 00:00:00,13.070000,13.280000,13.070000,13.250000,210300
4,"Exponent, Inc.",Industrials,Consulting Services,EQUITY,None,"Exponent, Inc., together with its subsidiaries...",EXPO,2020-12-04 00:00:00,81.949997,83.300003,81.949997,83.099998,193800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,"MiMedx Group, Inc.",Healthcare,Medical Devices,EQUITY,None,"MiMedx Group, Inc., a biopharmaceutical compan...",MDXG,2020-12-04 00:00:00,6.500000,6.600000,6.150000,6.190000,570300
2392,Bally's Corporation,Consumer Cyclical,Resorts & Casinos,EQUITY,None,Bally's Corporation owns and operates gaming a...,BALY,2020-12-04 00:00:00,45.200001,45.939999,43.520000,45.709999,360300
2393,"USA Technologies, Inc.",Technology,Software—Infrastructure,EQUITY,None,"USA Technologies, Inc. provides wireless netwo...",USAT,2020-12-04 00:00:00,8.770000,9.810000,8.770000,9.440000,291600
2394,"Wrap Technologies, Inc.",Technology,Scientific & Technical Instruments,EQUITY,None,"Wrap Technologies, Inc., a security technology...",WRAP,2020-12-04 00:00:00,5.350000,5.625000,5.350000,5.530000,336000


In [41]:
beeps()

In [5]:
get_df_prices('BYND', '2020-11-28', '2020-12-03')

,sym,datetime,open,high,low,adj_close,volume,is_reg_hours
133,BYND,2020-11-30 05:21:00,141.900000,141.900000,141.900000,141.900000,0,0
134,BYND,2020-11-30 07:07:00,140.360000,140.360000,140.350000,140.350000,0,0
135,BYND,2020-11-30 08:17:00,140.530000,140.530000,140.530000,140.530000,0,0
136,BYND,2020-11-30 08:18:00,140.900000,140.900000,140.900000,140.900000,0,0
137,BYND,2020-11-30 08:33:00,141.180000,141.250000,141.180000,141.250000,0,0
...,...,...,...,...,...,...,...,...
1066,BYND,2020-12-02 10:50:00,135.070007,135.388702,134.800003,135.070007,11643,1
1067,BYND,2020-12-02 10:51:00,134.860001,135.559998,134.800003,135.490005,14073,1
1068,BYND,2020-12-02 10:52:00,135.489899,135.930206,135.445007,135.930206,10986,1
1069,BYND,2020-12-02 10:53:00,135.759995,136.339996,135.759995,136.294998,14608,1


In [19]:
dt_info = yf.Ticker('CDNA').info